# Using the Materials API

We will cover:

* getting an API key
* `pymatgen.MPRester` built-ins
* `MPRester.query`
* the [mapidoc](https://github.com/materialsproject/mapidoc) repository
* accessing the API more directly, with plain HTTP requests (via the Python `requests` library)

## Getting an API key

Go to https://materialsproject.org/dashboard. If you see 'None' under the 'Generate API key' button, click the button. Highlight and copy the string under the button -- that is your API key. Paste it in as the value inside the quotation marks ("") below.

In [ ]:
import os
os.environ['MAPI_KEY'] = "YOUR_API_KEY"

## Creating a phase diagram

In [ ]:
%matplotlib inline

from pymatgen import MPRester
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.phasediagram.analyzer import PDAnalyzer
from pymatgen.phasediagram.maker import PhaseDiagram
from pymatgen.phasediagram.plotter import PDPlotter

In [ ]:
system = ["Fe", "P"]

In [ ]:
mpr = MPRester() # object for connecting to MP REST interface

Set energy corrections and +U/pseudopotential choice:

In [ ]:
compat = MaterialsProjectCompatibility()

Create phase diagram by fetching unprocessed entries from MP,

In [ ]:
unprocessed_entries = mpr.get_entries_in_chemsys(system)

filtering and adding energy corrections,

In [ ]:
processed_entries = compat.process_entries(unprocessed_entries)

and, finally, instantiating a Phase Diagram object from thge processed entries:

In [ ]:
pd = PhaseDiagram(processed_entries)

Let's plot the phase diagram.

In [ ]:
plotter = PDPlotter(pd, show_unstable=0)
plotter.show()

In [ ]:
plotter = PDPlotter(pd, show_unstable=0.100)
plotter.show()

Let's analyze the phase diagram to list stable and unstable entries.

In [ ]:
pda = PDAnalyzer(pd)

A table of stable entries:

In [ ]:
for e in pd.stable_entries:
    print("{}\t{}".format(e.composition.reduced_formula, e.entry_id))

Now, let's print a table of unstable entries, inlcuding their energy above hull and predicted decomposition. Since this is more involved display-wise, we'll use a handy and popular library for dealing with data frames and displaying them nicely inline.

In [ ]:
from pandas import DataFrame

In [ ]:
data = []
for e in pd.unstable_entries:
    decomp, e_above_hull = pda.get_decomp_and_e_above_hull(e)
    pretty_decomp = [
        ("{}:{}".format(k.composition.reduced_formula, k.entry_id), round(v, 2))
        for k, v in decomp.items()]
    data.append({
            'formula': e.composition.reduced_formula,
            'material_id': e.entry_id,
            'e_above_hull (eV/atom)': "{:.3f}".format(e_above_hull),
            'decomposes_to': pretty_decomp
            })
df = DataFrame(data,
               columns=["formula", "material_id", "e_above_hull (eV/atom)", "decomposes_to"])

print("Unstable Entries:")
df

In [ ]:
mpr.query('mp-27164', ['e_above_hull'])